In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, timestampdiff
import os

spark = SparkSession.builder.appName("EmployeeAttendanceCleaning").getOrCreate()

delta_table_path = "/delta/attendance_data"

try:
    attendance_df = spark.read.format("delta").load(delta_table_path)

    cleaned_attendance_df = attendance_df.na.drop(subset=["CheckInTime,CheckOutTime"])

    cleaned_attendance_df = cleaned_attendance_df.withColumn("HoursWorked",
    timestampdiff("HOUR",col("CheckInTime"), col("CheckOutTime")))

    cleaned_attendance_df.write.format("delta").mode("overwrite").save("/delta/attendance_data_cleaned")

except AnalysisException:
    print("Delta table not found")
except Exception as e:
    print(f"An error occurred: {str(e)}")


